In [1]:
# 1) design model (input, output size, forward pass)
# 2) construct loss and optimizer
# 3) training loop (forward pass, backward pass, update weights)

In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [5]:
# 0) prepare data

bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=1234)

#scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

In [8]:
# 1) model
# f = wx +b, sigmoid at end

class LogisticRegression(nn.Module):

  def __init__(self, n_input_features):
    super(LogisticRegression, self).__init__()
    self.linear = nn.Linear(n_input_features,1)

  def forward(self,x):
    y_predicted = torch.sigmoid(self.linear(x))
    return y_predicted

model = LogisticRegression(n_features)

# 2) loss and optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr =learning_rate)

# 3) training loop
num_epochs = 100

for epoch in range(num_epochs):
  y_predicted = model(X_train)
  loss = criterion(y_predicted,y_train)

  loss.backward()

  optimizer.step()

  optimizer.zero_grad()

  if (epoch +1) % 10 ==0:
    print(f"epoch {epoch+1}: loss = {loss:.4f}")

with torch.no_grad():
  y_predicted = model(X_test)
  y_predicted_cls = y_predicted.round()
  acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
  print(f"accuracy = {acc:.4f}")


epoch 10: loss = 0.5887
epoch 20: loss = 0.4892
epoch 30: loss = 0.4240
epoch 40: loss = 0.3777
epoch 50: loss = 0.3431
epoch 60: loss = 0.3160
epoch 70: loss = 0.2941
epoch 80: loss = 0.2761
epoch 90: loss = 0.2609
epoch 100: loss = 0.2479
accuracy = 0.9035
